In [4]:
# http://blog.districtdatalabs.com/data-exploration-with-python-2

import os
import zipfile
import requests

import pandas as pd

In [2]:
def download_data(url, name, path='data'):
    if not os.path.exists(path):
        os.mkdir(path)

    response = requests.get(url)
    with open(os.path.join(path, name), 'wb') as f:
        f.write(response.content)

    z = zipfile.ZipFile(os.path.join(path, 'vehicles.zip'))
    z.extractall(path)

In [3]:
VEHICLES = 'http://bit.ly/ddl-cars'

download_data(VEHICLES, 'vehicles.zip')

In [6]:
path = 'data'
vehicles = pd.read_csv(os.path.join(path, 'vehicles.csv'))
vehicles.head()

C:\Users\Orhan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [12]:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive', 'VClass',
                  'fuelType','barrels08', 'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

vehicles = vehicles[select_columns][vehicles.year <= 2016].drop_duplicates().dropna()

vehicles = vehicles.sort_values(['make', 'model', 'year'])

vehicles.columns = ['Make','Model','Year','Engine Displacement','Cylinders',
                    'Transmission','Drivetrain','Vehicle Class','Fuel Type',
                    'Fuel Barrels/Year','City MPG','Highway MPG','Combined MPG',
                    'CO2 Emission Grams/Mile','Fuel Cost/Year']

In [14]:
print(vehicles.shape)
vehicles.head()

(35095, 15)


,Make,Model,Year,Engine Displacement,Cylinders,Transmission,Drivetrain,Vehicle Class,Fuel Type,Fuel Barrels/Year,City MPG,Highway MPG,Combined MPG,CO2 Emission Grams/Mile,Fuel Cost/Year
19316,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,19.388824,18,17,17,522.764706,1950
19314,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
358,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,20.600625,16,17,16,555.437500,2100
369,AM General,Post Office DJ8 2WD,1985,4.2,6.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
25797,ASC Incorporated,GNX,1987,3.8,6.0,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,20.600625,14,21,16,555.437500,2550


## Aggregating to Higher-Level Categories

In [21]:
def unique_col_values(df):
    for column in df:
        print("{} | {} | {}".format(
            column, len(df[column].unique()), df[column].dtype))
        
unique_col_values(vehicles)

Make | 126 | object
Model | 3490 | object
Year | 33 | int64
Engine Displacement | 65 | float64
Cylinders | 9 | float64
Transmission | 44 | object
Drivetrain | 7 | object
Vehicle Class | 34 | object
Fuel Type | 13 | object
Fuel Barrels/Year | 116 | float64
City MPG | 48 | int64
Highway MPG | 49 | int64
Combined MPG | 46 | int64
CO2 Emission Grams/Mile | 550 | float64
Fuel Cost/Year | 55 | int64


In [23]:
vehicles['Transmission'].value_counts()

Automatic 4-spd                     10582
Manual 5-spd                         7764
Automatic 3-spd                      2597
Automatic (S6)                       2455
Manual 6-spd                         2336
Automatic 5-spd                      2171
Automatic 6-spd                      1351
Manual 4-spd                         1306
Automatic (S5)                        822
Automatic (S8)                        774
Automatic (variable gear ratios)      643
Automatic 7-spd                       630
Automatic (S7)                        238
Automatic (S4)                        229
Auto(AM-S7)                           211
Automatic 8-spd                       207
Auto(AV-S6)                           135
Auto(AM7)                             124
Auto(AM6)                             103
Auto(AM-S6)                            82
Manual 3-spd                           74
Manual 7-spd                           56
Auto(AV-S7)                            54
Automatic 9-spd                   

In [24]:
AUTOMATIC = "Automatic"
MANUAL = "Manual"

vehicles.loc[vehicles['Transmission'].str.startswith('Auto'),'Transmission Type'] = AUTOMATIC
vehicles.loc[vehicles['Transmission'].str.startswith('Man'),'Transmission Type'] = MANUAL

In [26]:
vehicles['Transmission Type'].value_counts()

Automatic    23556
Manual       11539
Name: Transmission Type, dtype: int64

In [27]:
vehicles['Vehicle Class'].value_counts()

Compact Cars                          5048
Subcompact Cars                       4273
Midsize Cars                          3951
Standard Pickup Trucks                2311
Sport Utility Vehicle - 4WD           2081
Two Seaters                           1724
Large Cars                            1678
Sport Utility Vehicle - 2WD           1615
Special Purpose Vehicles              1404
Small Station Wagons                  1344
Minicompact Cars                      1165
Standard Pickup Trucks 2WD            1125
Vans                                  1121
Standard Pickup Trucks 4WD             938
Midsize-Large Station Wagons           633
Special Purpose Vehicle 2WD            570
Small Pickup Trucks                    524
Vans, Cargo Type                       432
Midsize Station Wagons                 413
Small Sport Utility Vehicle 4WD        412
Small Pickup Trucks 2WD                398
Standard Sport Utility Vehicle 4WD     343
Minivan - 2WD                          324
Small Sport

In [36]:
d = {}

d['Small Cars'] = ['Compact Cars','Subcompact Cars','Two Seaters','Minicompact Cars']
d['Midsize Cars'] = ['Midsize Cars']
d['Large Cars'] = ['Large Cars']

d

{'Large Cars': ['Large Cars'],
 'Midsize Cars': ['Midsize Cars'],
 'Small Cars': ['Compact Cars',
  'Subcompact Cars',
  'Two Seaters',
  'Minicompact Cars']}

In [37]:
for k, v in d.items():
    vehicles.loc[vehicles['Vehicle Class'].isin(v), 'Vehicle Category'] = k

In [39]:
vehicles['Vehicle Category'].value_counts()

Small Cars      12210
Midsize Cars     3951
Large Cars       1678
Name: Vehicle Category, dtype: int64

In [43]:
vehicles['Model']

19316      DJ Po Vehicle 2WD
19314       FJ8c Post Office
358      Post Office DJ5 2WD
369      Post Office DJ8 2WD
25797                    GNX
3678             2.2CL/3.0CL
3680             2.2CL/3.0CL
3681             2.2CL/3.0CL
4488             2.3CL/3.0CL
4489             2.3CL/3.0CL
4491             2.3CL/3.0CL
5372             2.3CL/3.0CL
5373             2.3CL/3.0CL
5374             2.3CL/3.0CL
1989                   2.5TL
2972             2.5TL/3.2TL
2973             2.5TL/3.2TL
3803             2.5TL/3.2TL
3804             2.5TL/3.2TL
4627             2.5TL/3.2TL
4628             2.5TL/3.2TL
7304                   3.2CL
8321                   3.2CL
9398                   3.2CL
9399                   3.2CL
5633                   3.2TL
6524                   3.2TL
7483                   3.2TL
8516                   3.2TL
9588                   3.2TL
                ...         
20834              Wetl 850I
20835              Wetl 850I
35063        Wetl Testarossa
20826        W

In [ ]:
# Next, let's look at the Make and Model fields